In [28]:
from dataset import Dataset
import time
import scipy.io as scio
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split  
from sklearn.svm import SVC  
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.metrics.classification import cohen_kappa_score
from sklearn.metrics import accuracy_score

In [29]:
def get_wts(arr):
    return norm.cdf(arr,np.mean(arr),np.std(arr))

bots=Dataset("Botswana",".mat")
bots.load()
hypd=bots.data


shp=np.shape(hypd)
row=shp[0]
col=shp[1]
bands=shp[2]
    
hypd=hypd.reshape(row*col,bands)
    
bots_gt=Dataset('Botswana_gt','.mat')
bots_gt.load()
y=bots_gt.data.reshape(row*col,1)
ind=np.where(y[:,0]!=0)
X_hypd=hypd[ind]
y = y[y!=0]

In [30]:
df = pd.DataFrame(X_hypd)

df2=pd.DataFrame()

band=dict()
band['Blue']=np.linspace(490,530,5)
band['Green']=np.linspace(530,590,7)
band['Red']=np.linspace(630,680,6)
band['NIR']=np.linspace(850,880,4)



wblue=np.ediff1d(get_wts(band['Blue']))
wgreen=np.ediff1d(get_wts(band['Green']))
wred=np.ediff1d(get_wts(band['Red']))
wnir=np.ediff1d(get_wts(band['NIR']))


df2['Blue'] = (df.iloc[:, 0:4].multiply(wblue, axis=1).sum(axis=1))/np.sum(wblue)
df2['Green'] = (df.iloc[:, 4:10].multiply(wgreen, axis=1).sum(axis=1))/np.sum(wgreen)
df2['Red'] = (df.iloc[:, 14:19].multiply(wred, axis=1).sum(axis=1))/np.sum(wred)
df2['NIR'] = (df.iloc[:, 36:39].multiply(wnir, axis=1).sum(axis=1))/np.sum(wnir)

In [45]:
X_train, X_test, y_train, y_test = train_test_split(df2, y, test_size=0.33)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()  
sc.fit_transform(X_train)
Xn_train = sc.transform(X_train)
Xn_test = sc.transform(X_test)

# Grid Search to find best set of parameters: OVR

In [46]:
from sklearn.model_selection import GridSearchCV

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
svm=SVC()
#scores = cross_val_score(svm, X_pc3, y, cv=5)
clf = GridSearchCV(svm, tuned_parameters, cv=5,)
clf.fit(Xn_train, y_train)

print("Best parameters set found:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found:

{'C': 100, 'kernel': 'linear'}

Grid scores on development set:

0.299 (+/-0.014) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.097 (+/-0.002) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.558 (+/-0.028) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.297 (+/-0.016) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.785 (+/-0.027) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.558 (+/-0.028) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.847 (+/-0.019) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.785 (+/-0.027) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.830 (+/-0.014) for {'C': 1, 'kernel': 'linear'}
0.858 (+/-0.012) for {'C': 10, 'kernel': 'linear'}
0.860 (+/-0.017) for {'C': 100, 'kernel': 'linear'}
0.860 (+/-0.019) for {'C': 1000, 'kernel': 'linear'}


In [48]:
C=clf.best_params_['C']
if(clf.best_params_['kernel']=='rbf'):
    gamma=clf.best_params_['gamma']
    svm=SVC(kernel='rbf',C=C,gamma=gamma)
else:
    svm=SVC(kernel='linear',C=C)

start_time = time.time()
svm.fit(Xn_train,y_train)
y_pred=svm.predict(Xn_test)
acc=accuracy_score(y_test, y_pred)
print("Accuracy: %0.2f" % acc)
kappa_score = cohen_kappa_score(y_test,y_pred)
print("Kappa coefficient: %0.2f" % kappa_score)
print(confusion_matrix(y_test,y_pred))
print("--- %s seconds ---" % (time.time() - start_time))

In [35]:
from sklearn.model_selection import GridSearchCV

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
svm=SVC()
#scores = cross_val_score(svm, X_pc3, y, cv=5)
clf = GridSearchCV(svm, tuned_parameters, cv=5,)
clf.fit(X_train, y_train)

print("Best parameters set found:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found:

{'C': 100, 'kernel': 'linear'}

Grid scores on development set:

0.309 (+/-0.022) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.093 (+/-0.001) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.555 (+/-0.008) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.310 (+/-0.023) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.789 (+/-0.029) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.556 (+/-0.007) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.847 (+/-0.043) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.789 (+/-0.029) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.830 (+/-0.035) for {'C': 1, 'kernel': 'linear'}
0.857 (+/-0.031) for {'C': 10, 'kernel': 'linear'}
0.864 (+/-0.034) for {'C': 100, 'kernel': 'linear'}
0.860 (+/-0.038) for {'C': 1000, 'kernel': 'linear'}


In [36]:
C=clf.best_params_['C']
if(clf.best_params_['kernel']=='rbf'):
    gamma=clf.best_params_['gamma']
    svm=SVC(kernel='rbf',C=C,gamma=gamma)
else:
    svm=SVC(kernel='linear',C=C)

In [37]:
start_time = time.time()
svm.fit(X_train,y_train)
y_pred=svm.predict(X_test)
acc=accuracy_score(y_test, y_pred)
print("Accuracy: %0.2f" % acc)
kappa_score = cohen_kappa_score(y_test,y_pred)
print("Kappa coefficient: %0.2f" % kappa_score)
print(confusion_matrix(y_test,y_pred))
print("--- %s seconds ---" % (time.time() - start_time))

Accuracy: 0.87
Kappa coefficient: 0.86
[[86  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 35  0  0  2  0  0  0  0  0  0  0  0  0]
 [ 0  0 79  1  0  0  1  0  0  3  0  0  0  0]
 [ 0  0  5 66  0  0  0  0  0  0  0  0  0  0]
 [ 0  5  0  4 80  5  0  0  0  0  0  0  0  0]
 [ 0  4  0  0 14 47  0  0 28  0  0  0  0  0]
 [ 0  0  0  1  0  0 69  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 69  0  0  4  0  0  0]
 [ 0  0  0  0  1 21  0  0 89  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 49  7  0  2  0]
 [ 0  0  0  0  0  0  0  3  0 10 99  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 59  4  0]
 [ 0  0  0  0  0  0  0  0  2  3  1  3 78  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 32]]
--- 0.1093282699584961 seconds ---


# Grid Search to find best set of parameters: OVO

In [49]:
from sklearn.model_selection import GridSearchCV

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
svm=SVC(decision_function_shape='ovo')
#scores = cross_val_score(svm, X_pc3, y, cv=5)
clf = GridSearchCV(svm, tuned_parameters, cv=5,)
clf.fit(Xn_train, y_train)

print("Best parameters set found:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found:

{'C': 100, 'kernel': 'linear'}

Grid scores on development set:

0.299 (+/-0.014) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.097 (+/-0.002) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.558 (+/-0.028) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.297 (+/-0.016) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.785 (+/-0.027) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.558 (+/-0.028) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.847 (+/-0.019) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.785 (+/-0.027) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.830 (+/-0.014) for {'C': 1, 'kernel': 'linear'}
0.858 (+/-0.012) for {'C': 10, 'kernel': 'linear'}
0.860 (+/-0.017) for {'C': 100, 'kernel': 'linear'}
0.860 (+/-0.019) for {'C': 1000, 'kernel': 'linear'}


In [50]:
C=clf.best_params_['C']
if(clf.best_params_['kernel']=='rbf'):
    gamma=clf.best_params_['gamma']
    svm=SVC(kernel='rbf',C=C,gamma=gamma,decision_function_shape='ovo')
else:
    svm=SVC(kernel='linear',C=C,decision_function_shape='ovo')

In [52]:
start_time = time.time()
svm.fit(Xn_train,y_train)
y_pred=svm.predict(Xn_test)
acc=accuracy_score(y_test, y_pred)
print("Accuracy: %0.2f" % acc)
kappa_score = cohen_kappa_score(y_test,y_pred)
print("Kappa coefficient: %0.2f" % kappa_score)
print(confusion_matrix(y_test,y_pred))
print("--- %s seconds ---" % (time.time() - start_time))

Accuracy: 0.87
Kappa coefficient: 0.86
[[83  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 23  0  0  2  0  0  0  0  0  0  0  0  0]
 [ 0  0 75  3  0  0  3  1  0  2  0  0  0  0]
 [ 0  0  4 64  5  0  0  0  0  1  0  0  0  0]
 [ 0  3  0  3 71  4  0  1  1  0  0  1  0  0]
 [ 0  2  0  0 10 47  0  0 23  0  0  0  0  0]
 [ 0  0  3  1  0  0 95  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 58  0  0  4  0  0  0]
 [ 0  0  0  0  0 14  0  0 89  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  2  1 74  9  0  1  0]
 [ 0  0  0  0  0  0  0  3  0  8 98  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 51  4  0]
 [ 0  0  0  0  0  1  0  0  3  4  2  4 66  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 43]]
--- 0.11999058723449707 seconds ---


In [53]:
svm

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)